In [1]:
import pandas as pd 
import numpy as np
import math 
from matplotlib import pyplot

In [2]:
# Function: calculate MAD saccade
def at_mad(angular_vel, th_0=200):
    # defines the saccade threshold (code from Ashima)
    threshs = []
    while True:
        # take th_0
        threshs.append(th_0)
        # get all angles smaller than this
        angular_vel = angular_vel[angular_vel < th_0]

        # MAD:
        # take the median of all angles smaller than th_0
        median = np.median(angular_vel)
        # substract the median value
        diff = np.sqrt((angular_vel - median) ** 2)
        # get the median of these values
        med_abs_deviation = np.median(diff)

        # calcualte the next threshold with the median
        # 1.486 used when assuming a normal distribution
        th_1 = median + 3 * 1.486 * med_abs_deviation
        # if the thresholds are too different, redo the while loop
        if abs(th_0 - th_1) > 1:
            th_0 = th_1
        # else, set the final threshold to the current one, break the while loop and return values
        else:
            saccade_thresh = th_1
            threshs.append(saccade_thresh)
            break
    return saccade_thresh, threshs

In [3]:
One_participant = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/04_Interpolated/0479_2.csv" )

In [4]:
#Complete = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/Complete_Data_AngularV.cvs")

In [5]:
#One= Complete[(Complete.SubjectID == 479) & (Complete.Session == 2)].copy()

In [12]:

######## Debbies Algorithm ########

for_eye = One_participant.copy()
time = for_eye["timeStampDataPointEnd"].tolist()

## Calculate Angular velocities
# get individual coordinates
subj = list(zip( for_eye["eyePositionCombinedWorld.x"],for_eye["eyePositionCombinedWorld.y"],for_eye["eyePositionCombinedWorld.z"]))
hpoo = list(zip(for_eye["hitPointOnObject_x"], for_eye["hitPointOnObject_y"],for_eye["hitPointOnObject_z"]))
# v_gaze_vec: get difference in hpoo
v_gaze_vec = list(zip(for_eye["hitPointOnObject_x"].diff(), for_eye["hitPointOnObject_y"].diff(),for_eye["hitPointOnObject_z"].diff()))
# get difference in time:
ts = for_eye["timeStampDataPointEnd"].diff().tolist()
# gaze_vec(t) is a unit vector in the direction of the gaze (eye+head) in world coordinates
g_vec = list(np.subtract(hpoo, subj))
gaze_vec = [np.array(v) / np.linalg.norm(np.array(v)) for v in g_vec]
# v_gaze_inplane: is a scalar indicating the velocity in world coordinates at the location that is gazed at orthogonal to the gaze axis.
z1 = [np.dot(v_gaze_vec_i, gaze_vec_i) for v_gaze_vec_i, gaze_vec_i in zip(v_gaze_vec, gaze_vec)]
# z = (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))
z = [z1[element] * np.array(gaze_vec[element]) for element in range(len(z1))]
# ||v_gaze_vec(t) - z||
v_gaze_inplane = np.linalg.norm(np.array(v_gaze_vec) - z, axis=-1)
#Eucledian distance between eye coordinates and hit on object
sub_hpoo = np.linalg.norm(np.array(subj) - np.array(hpoo), axis=-1)
# arctan2(v_gaze_inplane, sub_hpoo)
w_gaze = np.arctan2(v_gaze_inplane, sub_hpoo).tolist()
# Turn angle of radians into degrees over seconds 
w_gaze = [(w / ts[idx] * 180 / math.pi) for idx, w in enumerate(w_gaze)]
# save df
for_eye["combined_vel"] = w_gaze

### 10 second for threshold calculation starts here

int_len = 10  # number of seconds of the interval
time = for_eye["timeStampDataPointEnd"].values
start = [time[0]]
end = []
start_idx = [0]
end_idx = []
for t, ti in enumerate(time[1:], start=1):
    if ti - start[-1] > int_len:
        end.append(time[t - 1])
        end_idx.append(t - 1)
        start.append(ti)
        start_idx.append(t)
# add the last timepoint to end
end.append(time[-1])
end_idx.append(len(time))

# save it as new df
int_data = pd.DataFrame({
    "start": start,
    "end": end,
    "start_idx": start_idx,
    "end_idx": end_idx
})

combined_vel = for_eye["combined_vel"]

# to add the final thresholds to for each segement
scct = []
for s, srt in enumerate(int_data["start"].values):
    # get the slice of the combined velocity
    angular_vel = combined_vel[start_idx[s] : end_idx[s]]
    # use the at_mad function to caluclate the threshold
    saccade_th, thres = at_mad(angular_vel)
    if np.isnan(saccade_th):
        scct.append(thres[0])
    else:
        # add it to scct
        scct.append(saccade_th)

# add it to int_data and save
int_data["thresh"] = scct

ranges = list(zip(int_data.start_idx, int_data.end_idx))    
# go through all time intervals repeat the threshold as often as the time interval is long
for i, (lower, upper) in enumerate(ranges):
    for_eye.loc[lower:upper,"thresh"]  = int_data["thresh"][i]
    
# go through combined velocity and save all that are bigger than the threshold
    # Everywhere  where there is Nans that is a saccade meaning this are the cells that are really fast OR ####### we had nan on the combined velocity #######

for_eye["isFix"] = np.where(for_eye['combined_vel'] < for_eye["thresh"], for_eye['combined_vel'], np.nan)
 ### reset for following parts
    # specify the variables you want to delete in a list
    
to_delete = [int_data, start, end, start_idx, end_idx]

# delete the variables using a loop
for Object in to_delete:
    del Object



In [14]:
%%timeit
for_eye.reset_index(inplace=True, drop=True)

min_sacc_dur = 0.02  # min sacc duration
min_gaze_dur = 0.04  # min gaze duration (Ashima uses 0.05)
time = for_eye.timeStampDataPointEnd.values
index = for_eye.index.tolist()  # index of df for easier use
start_time = time[0]  # update for each change
start_idx = index[0]  # will be updated each event and used to add to the lists

# to save:
isFix = []
combined_vel = []

# if the first sample does not have any data
if pd.isna(for_eye.loc[0, "combined_vel"]) and not pd.isna(for_eye.loc[1, "combined_vel"]):
    start_time = time[1]   # update for each change
    start_idx = index[1]  # will be updated each event and used to add to the lists
    isFix = [np.nan]
    combined_vel = [np.nan]

7.07 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
#%%timeit
min_sacc_dur = 0.02  # min sacc duration
min_gaze_dur = 0.04  # min gaze duration (Ashima uses 0.05)
time = for_eye.timeStampDataPointEnd.values
n_rows = for_eye.shape[0]  # number of rows in the DataFrame

# to save:
isFix = [np.nan] * n_rows
combined_vel = [np.nan] * n_rows

# if the first sample does not have any data
if pd.isna(for_eye.iloc[0, "combined_vel"]) and not pd.isna(for_eye.iloc[1, "combined_vel"]):
    start_time = time[1]   # update for each change
    start_idx = 1  # will be updated each event and used to add to the lists
else:
    start_time = time[0]
    start_idx = 0

for i in range(n_rows):
    if pd.isna(for_eye.iloc[i, "combined_vel"]):
        isFix[i] = np.nan
        combined_vel[i] = np.nan
    else:
        if time[i] - start_time >= min_gaze_dur:
            if i - start_idx == 1:
                isFix[start_idx] = False
            else:
                isFix[start_idx:i] = True
            combined_vel[start_idx:i] = for_eye.iloc[start_idx:i, "combined_vel"].mean()
            start_time = time[i]
            start_idx = i

# if the last sample is part of a fixation
if isFix[-1] == np.nan:
    if n_rows - start_idx == 1:
        isFix[start_idx] = False
    else:
        isFix[start_idx:] = True
    combined_vel[start_idx:] = for_eye.iloc[start_idx:, "combined_vel"].mean()

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [ ]:
%timeit pd.isna(for_eye.loc[0, "combined_vel"])

In [ ]:
%timeit math.isnan(for_eye.iloc[0]["combined_vel"])

In [ ]:
math.isnan(for_eye.iloc[0]["combined_vel"])

In [ ]:
pd.isna(for_eye.loc[0, "combined_vel"])

In [ ]:
for_eye.loc[2, 'isFix']

In [ ]:
for_eye["CheckisFix"] = for_eye["isFix"] == for_eye['isFixi']

In [ ]:
combined = pd.DataFrame(combined_vel == for_eye["combined_vel"])

In [ ]:
combined = pd.DataFrame(combined_vel != for_eye["combined_vel"])

In [ ]:
len(av)
len(av)

In [ ]:
len(start)

In [ ]:
len(ranges)

In [ ]:
for_eye["FIXICHECK"] = for_eye['fixi'] != for_eye["isFix"]

In [ ]:
com = pd.DataFrame(av == avi)

In [ ]:
for_eye["FIXICHECK"].value_counts(normalize=True)

In [ ]:
show = for_eye[for_eye.FIXICHECK]

In [ ]:
show.head(50)

In [ ]:

for i, (lower,upper) in enumerate(ranges):
    avi = for_eye.loc[lower:upper, "combined_vel"].values
    fixi = [item if item < int_data["thresh"][i] else np.nan for item in avi]
    for_eye.loc[lower:upper,"isFixi"]  = for_eye.loc[]

In [ ]:
len(avi)

In [ ]:
len(fixi)

In [ ]:
for_eye['fixi'] = np.where(for_eye['combined_vel'] < for_eye["thresh"], for_eye['combined_vel'], np.nan)

In [ ]:
for_eye['fixi']

In [ ]:
# eye-tracking
thres = int_data["thresh"].values
combined_vel = for_eye["combined_vel"].values

# define list where the fixations will be added too
is_fix = [np.nan] * len(combined_vel)
start = int_data["start_idx"].tolist()
end = int_data["end_idx"].tolist()

# eye-tracking
thres = int_data["thresh"].tolist()
combined_vel = for_eye["combined_vel"].tolist()
for i in range(len(start)):
    av = combined_vel[start[i] : end[i]]

In [ ]:
for_eye.tail()

In [ ]:
len(start)

In [ ]:

for i, (lower, upper) in enumerate(ranges):
    for_eye.loc[lower:upper,"is-FIXI"]  = int_data["thresh"][i]

In [ ]:
# eye-tracking
thres = int_data["thresh"].values
combined_vel = for_eye["combined_vel"].values

# define list where the fixations will be added too
is_fix = [np.nan] * len(combined_vel)
start = int_data["start_idx"].tolist()
end = int_data["end_idx"].tolist()

# eye-tracking
thres = int_data["thresh"].tolist()
combined_vel = for_eye["combined_vel"].tolist()


for i in range(len(start)):
    av = combined_vel[start[i] : end[i]]
    # go through combined velocity and save all that are bigger than the threshold
    fix = [ti if ti < thres[i] else np.nan for ti in av]
    is_fix[start[i] : end[i]] = fix
for_eye["isFix"] = is_fix

In [ ]:
%%timeit

######## Debbies Algorithm ########

for_eye = One_participant.copy()

time = for_eye["timeStampDataPointEnd"].tolist()
# get individual coordinates
subj = list(zip( for_eye["eyePositionCombinedWorld.x"],for_eye["eyePositionCombinedWorld.y"],for_eye["eyePositionCombinedWorld.z"]))
hpoo = list(zip(for_eye["hitPointOnObject_x"], for_eye["hitPointOnObject_y"],for_eye["hitPointOnObject_z"]))

# v_gaze_vec: get difference in hpoo
v_vX = for_eye["hitPointOnObject_x"].diff()
v_vY = for_eye["hitPointOnObject_y"].diff()
v_vZ = for_eye["hitPointOnObject_z"].diff()


# get difference in time:
ts = pd.DataFrame(time).apply(lambda x: x.diff())[0].tolist()
v_gaze_vec = list(zip(v_vX, v_vY, v_vZ))

# gaze_vec(t) is a unit vector in the direction of the gaze (eye+head) in world coordinates
g_vec = [np.array(hpoo[v] - np.array(subj[v])) for v in range(len(subj))]
gaze_vec = [np.array(v) / np.linalg.norm(np.array(v)) for v in g_vec]


# v_gaze_inplane: is a scalar indicating the velocity in world coordinates at the location that is gazed at orthogonal to the gaze axis.
# v_gaze_inplane(t) = ||v_gaze_vec(t) - (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))||:
# z1 = (<v_gaze_vec(t), gaze_vec(t)>)
z1 = [np.array(v_gaze_vec[t]).dot(np.array(gaze_vec[t]))
        for t in range(len(v_vX))]

# z = (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))
z = [z1[t] * np.array(gaze_vec[t]) for t in range(len(v_vX))]

# ||v_gaze_vec(t) - z||
v_gaze_inplane = [np.linalg.norm((np.array(v_gaze_vec[t]) - z[t]).tolist())
        for t in range(len(v_gaze_vec))
                 ]
# w_gaze(t) = arctan2(||subject_vec(t) - hpoo_vec(t)||, v_gaze_inplane)

# sub_hpoo = ||subject_vec(t) - hpoo_vec(t)||
sub_hpoo = [np.linalg.norm(np.array(subj[t]) - np.array(hpoo[t]))
        for t in range(len(hpoo))]

# arctan2(v_gaze_inplane, sub_hpoo)
w_gaze = np.arctan2(v_gaze_inplane, sub_hpoo).tolist()

# turn angle of radians into degrees
#w_gaze = [np.rad2deg(value) for value in w_gaze]
w_gaze = [(w / ts[idx] * 180 / math.pi) for idx, w in enumerate(w_gaze)]
# save df
for_eye["combined_vel"] = w_gaze
### 10 second starts here 
int_len = 10  # number of seconds of the interval

time = for_eye["timeStampDataPointEnd"].tolist()
start = []
end = []
start_idx = []
end_idx = []
for t, ti in enumerate(time):
    if ti == time[0]:
        start.append(ti)
        start_idx.append(t)
    if ti - start[-1] > int_len:
        # if the current timepoint is more than int_len away from start, set it to new start
        start.append(ti)
        start_idx.append(t)
        # and set end to the timepoint before that
        end.append(time[t - 1])
        end_idx.append(t - 1)
# add the last timepoint to end
# (there is a very slim chance that the last start and end are the same timepoint --> might cause errors)
end.append(time[-1])
end_idx.append(len(time))

# save it as new df
int_data = list(zip(start, end, start_idx, end_idx))
int_data = pd.DataFrame(
    int_data, columns=["start", "end", "start_idx", "end_idx"]
)

combined_vel = for_eye["combined_vel"]

# to shorten the slicing in the next for loop
start_idx = int_data["start_idx"].tolist()
end_idx = int_data["end_idx"].tolist()

# to add the final thresholds to for each segement
scct = []
for s, srt in enumerate(int_data["start"].tolist()):
    # get the slice of the combined velocity
    angular_vel = combined_vel[start_idx[s] : end_idx[s]]
    # use the at_mad function to caluclate the threshold
    saccade_th, thres = at_mad(angular_vel)
    if np.isnan(saccade_th):
        scct.append(thres[0])
    else:
        # add it to scct
        scct.append(saccade_th)

# add it to int_data and save
int_data["thresh"] = scct
int_data = pd.DataFrame(int_data)
time = for_eye.timeStampDataPointEnd.tolist()
start_idx = int_data["start_idx"].tolist()
end_idx = int_data["end_idx"].tolist()
thr = int_data["thresh"].tolist()

# go through all time intervals
thresh = [0.0] * len(time)
for s, srt in enumerate(int_data["start"].tolist()):
    # repeat the threshold as often as the time interval is long
    thresh = (
        thresh[: start_idx[s]]
        + [thr[s]] * len(time[start_idx[s] : end_idx[s]])
        + thresh[end_idx[s]:]
    )

# add the two lists (ht & et) to for_eye df
for_eye["thresh"] = thresh

# save for_eye df
for_eye = pd.DataFrame(for_eye)

start = int_data["start_idx"].tolist()
end = int_data["end_idx"].tolist()

# eye-tracking
thres = int_data["thresh"].tolist()
combined_vel = for_eye["combined_vel"].tolist()

# define list where the fixations will be added too
is_fix = [np.nan] * len(combined_vel)

for i in range(len(start)):
    av = combined_vel[start[i] : end[i]]
    # go through combined velocity and save all that are bigger than the threshold
    fix = [ti if ti < thres[i] else np.nan for ti in av]
    is_fix[start[i] : end[i]] = fix

# save
for_eye["isFix"] = is_fix


In [ ]:
ranges = list(zip(int_data.start_idx, int_data.end_idx))
#Replace the invalid event with the mode of 20 events prior
for i, (lower, upper) in enumerate(ranges):
    for_eye.loc[lower:upper,"THRESHI"]  = int_data["thresh"][i]

In [ ]:
for_eye.head(200)

In [ ]:
int_data.start

In [ ]:
for_eye.head()

In [ ]:
New = for_eye["combined_vel"]

In [ ]:
for_eye.columns

In [ ]:
Old = One["combined_vel"]

In [ ]:
pyplot.scatter(New, Old)

In [ ]:
from scipy.stats import spearmanr
corr, _ = spearmanr(New, Old, nan_policy='omit')
print('Spearmans correlation: %.3f' % corr)

In [ ]:
### 10 second starts here 
int_len = 10  # number of seconds of the interval
time = for_eye["timeStampDataPointEnd"].values
start = []
end = []
start_idx = []
end_idx = []
for t, ti in enumerate(time):
    if ti == time[0]:
        start.append(ti)
        start_idx.append(t)
    if ti - start[-1] > int_len:
        # if the current timepoint is more than int_len away from start, set it to new start
        start.append(ti)
        start_idx.append(t)
        # and set end to the timepoint before that
        end.append(time[t - 1])
        end_idx.append(t - 1)
# add the last timepoint to end
# (there is a very slim chance that the last start and end are the same timepoint --> might cause errors)
end.append(time[-1])
end_idx.append(len(time))

In [ ]:
for_eye.Continuous_Time.head(20)

In [ ]:
import datetime
for_eye["datetime_objects"] = [datetime.datetime.utcfromtimestamp(timestamp) for timestamp in for_eye["timeStampDataPointEnd"]]
df_resampled = for_eye.resample('10S', on='datetime_objects').first()
# Get the index of the resampled DataFrame
resampled_index = df_resampled.index

In [ ]:
df_resampled["saccade_thi"], df_resampled["thresi"] = for_eye["combined_vel"].rolling('10s').apply(at_mad)

In [ ]:
df_resampled["saccade_thi"], df_resampled["thresi"] = at_mad(df_resampled.combined_vel)

In [ ]:
saccade_thi

In [ ]:
df_resampled.columns

In [ ]:
len(df_resampled)

In [ ]:
import datetime
for_eye["datetime_objects"] = [datetime.datetime.utcfromtimestamp(timestamp) for timestamp in for_eye["timeStampDataPointEnd"]]
datetime.datetime.utcfromtimestamp(for_eye.loc[1,"timeStampDataPointEnd"])

In [ ]:
for_eye["time-time"]

In [ ]:
for_eye["Continuous_Time"].describe()

In [ ]:
# to shorten the slicing in the next for loop
start_idx = int_data["start_idx"].values
end_idx = int_data["end_idx"].values

# Vectorize calculation of angular velocity
angular_veli = np.array([combined_vel[start:end] for start, end in zip(start_idx, end_idx)])

# Calculate saccade thresholds for all segments at once
saccade_th, thres = at_mad(angular_vel)

# Replace NaN values with corresponding threshold from thres array
scct = []
scct.append(np.nan_to_num(thres)[0] if np.isnan(saccade_th) else saccade_th)

# Assign saccade thresholds to int_data DataFrame
int_data["threshi"] = scct

# Create a list of thresholds for each timestamp
time = for_eye.timeStampDataPointEnd.values
thresh = np.zeros(len(time))
for s, (start, end, th) in enumerate(zip(start_idx, end_idx, saccade_thi)):
    threshi[start:end] = th

# Add threshold column to for_eye DataFrame
for_eye["threshi"] = threshi


In [ ]:
scct = [at_mad(combined_vel[start_idx[s] : end_idx[s]])[0] if not np.isnan(at_mad(combined_vel[start_idx[s] : end_idx[s]])[0]) else at_mad(combined_vel[start_idx[s] : end_idx[s]])[1][0] for s, start in enumerate(int_data["start"])]

In [ ]:
int_data["threshi"] =  scct

In [ ]:
int_data

In [ ]:
len(angular_vel)

In [ ]:
len(angular_veli)

In [ ]:
start_idxi[-1]

In [ ]:
start_idx[-1]

In [ ]:
start_idx

In [ ]:
import numpy as np

INTERVAL_LENGTH = 10  # number of seconds of the interval
time = for_eye["timeStampDataPointEnd"].values
start = []
end = []
start_idx = []
end_idx = []
for t, ti in enumerate(time):
    if ti == time[0]:
        start.append(ti)
        start_idx.append(t)
    if ti - start[-1] > INTERVAL_LENGTH:
        # if the current timepoint is more than INTERVAL_LENGTH away from start, set it to new start
        start.append(ti)
        start_idx.append(t)
        # and set end to the timepoint before that
        end.append(time[t - 1])
        end_idx.append(t - 1)
# add the last timepoint to end
end.append(time[-1])
end_idx.append(len(time))

# save it as new df
int_data = {
    "start": start,
    "end": end,
    "start_idx": np.array(start_idx),
    "end_idx": np.array(end_idx),
}
int_data = pd.DataFrame(int_data)

combined_vel = for_eye["combined_vel"]

# to shorten the slicing in the next for loop
start_idx = int_data["start_idx"].tolist()
end_idx = int_data["end_idx"].tolist()

In [ ]:
INTERVAL_LENGTH = 10  # number of seconds of the interval

time = for_eye["timeStampDataPointEnd"].values
start_idxi = [0] + [idx+1 for idx, (t1, t2) in enumerate(zip(time[:-1], time[1:])) if t2 - t1 > INTERVAL_LENGTH]
starti = [time[idx] for idx in start_idx]
end_idxi = [idx for idx in start_idx[:-1]]
endi = [time[idx-1] for idx in end_idx] + [time[-1]]
#int_data = {
   ##

In [ ]:
start_idxi

In [ ]:
len(starti)

In [ ]:
end[-7]

In [ ]:
endi[-7]

In [ ]:
int_data

In [ ]:
df["diff"] = df["value"].rolling(window=2).apply(lambda x: x[1] - x[0])

In [ ]:
v_gaze_inplane1[5]

In [ ]:
sub_hpoo1 = np.linalg.norm(np.array(subj) - np.array(hpoo), axis=-1)

In [ ]:
sub_hpoo1[658]

In [ ]:
sub_hpoo[658]

In [ ]:
z2 = [np.dot(v_gaze_vec_i, gaze_vec_i) for v_gaze_vec_i, gaze_vec_i in zip(v_gaze_vec, gaze_vec)]

In [ ]:
z2[656]

In [ ]:
z1[656]

In [ ]:
for_eye["hitPointOnObject_x"].diff().tolist()[59]

In [ ]:
list(np.subtract(hpoo, subj))[142]

In [ ]:
g_vec[142]

In [ ]:

hpoo = list(zip(for_eye["hitPointOnObject_x"], for_eye["hitPointOnObject_y"],for_eye["hitPointOnObject_z"]))

In [ ]:
v_vX[59]

In [ ]:
g_vec

In [ ]:
path = "/Volumes/SSD/6258_PointingTask_Randomization_1666084297.70049.json"
with open(path, 'r') as file:
        # make json files parsable
        data = "[" + file.read()
        data = data[:len(data)-1] + "]"

        # read data per file
        subjectdf = pd.read_json(data)

        # insert participant id in every line
        subjectdf.insert(0, "SubjectID", [int(path[13:17])] * subjectdf.shape[0], True)

In [ ]:
subjectdf.head()

In [ ]:
StartingPoints = pd.DataFrame(subjectdf.explode('PointingTaskStartingLocations'))
StartingPoints.head()

In [ ]:
StartingPointsnorm = pd.json_normalize(subjectdf["InitialOrientation"]).add_prefix("InitialOrientation_")
StartingPointsnorm.head()

In [ ]:
%timeit -r 4 -n 1000 StartingPoints = pd.DataFrame(subjectdf.explode('PointingTaskStartingLocations'))
%timeit -r 4 -n 1000 StartingPointsnorm = pd.json_normalize(subjectdf["InitialOrientation"])

In [ ]:
subjectdf["PointingTaskRandomised"][0]

In [ ]:
normalized = pd.json_normalize(subjectdf["PointingTaskRandomised"][0])
normalized.head()

In [ ]:
Final = pd.DataFrame(normalized.explode('TargetLocationsRandom'))
Final.head()

In [ ]:
pd.json_normalize(subjectdf.PointingTaskStartingLocations")

In [ ]:
Dataset = pd.DataFrame()

In [ ]:
%%timeit -r 4 -n 1000
StartingPointsnorm = pd.json_normalize(subjectdf["InitialOrientation"]).add_suffix('_InitialOrientation')
StartingPointsnorm.head()